In [1]:
import os
import skvideo.io
import cv2
import shutil
import numpy as np
from tqdm import tqdm
from PIL import Image
import sys
import math
sys.path.append("../../../../CNN_total/")

def get_parent_dir_name(path, level=1):

    path_spliter = os.path.sep
    abs_path = os.path.abspath(path)

    return abs_path.split(path_spliter)[-(1 + level)]

In [2]:
from glob import glob

common_sample_folder = "../data/1. Sampling png"
common_split_folder = "../data/2. Split png"

video_class_list = sorted(os.listdir(common_sample_folder))
video_class_dict = {idx: class_str for idx, class_str in enumerate(video_class_list)}
video_class_instance_num_list = [len(glob(f"{common_sample_folder}/{video_class}/*"))
                                for video_class in video_class_list]

video_folder_list = glob(f"{common_sample_folder}/*/*")

print(f"video_class_num: {len(video_class_dict)}")
print(f"min_class_instance: {np.min(video_class_instance_num_list)}")
print(f"max_class_instance: {np.max(video_class_instance_num_list)}")
print(f"video_num: {len(video_folder_list)}")

frame_num_list = []
for video_folder in video_folder_list:
    frame_num = len(os.listdir(video_folder))
    frame_num_list.append(frame_num)
    
print(f"unique frame: {np.unique(frame_num_list)}")

video_class_num: 11
min_class_instance: 100
max_class_instance: 163
video_num: 1402
unique frame: [16]


In [3]:
train_ratio = 0.9
valid_ratio = 0.05
test_ratio = 0.05

for class_str in video_class_list:
    video_folder_list = glob(f"{common_sample_folder}/{class_str}/*")
    video_num = len(video_folder_list)
    train_num = math.ceil(video_num * train_ratio)
    valid_num = math.ceil(video_num * valid_ratio)
    
    for idx, (video_folder) in tqdm(enumerate(video_folder_list)):
        video_id = get_parent_dir_name(video_folder, level=0)
        
        if idx <= train_num:
            phase = "train"
        elif idx <= train_num + valid_num:
            phase = "valid"
        else:
            phase = "test"
        split_video_folder = f"{common_split_folder}/{phase}/{class_str}/{video_id}"
        shutil.copytree(video_folder, split_video_folder)


112it [00:07, 15.92it/s]
163it [00:10, 15.79it/s]
134it [00:08, 16.25it/s]
128it [00:07, 16.08it/s]
123it [00:07, 16.25it/s]
123it [00:07, 15.88it/s]
127it [00:07, 16.17it/s]
100it [00:06, 15.94it/s]
160it [00:10, 15.95it/s]
102it [00:06, 15.97it/s]
130it [00:08, 15.85it/s]


In [61]:
split_video_folder

'../data/2. Split png/train/BodyWeightSquats/v_BodyWeightSquats_g01_c01'

In [63]:
video_sample_path

'../data/1. Sampling png/BodyWeightSquats\\v_BodyWeightSquats_g01_c01\\000'

In [62]:
target_folder

'../data/2. Split png/train/BodyWeightSquats/v_BodyWeightSquats_g01_c01/000'

In [58]:
target_folder

'../data/2. Split png/train/BodyWeightSquats/v_BodyWeightSquats_g01_c01/000'

In [55]:
sample_id

'000'

In [53]:
video_id

'v_BodyWeightSquats_g01_c01'

In [51]:
video_sample_path

'../data/1. Sampling png/BodyWeightSquats\\v_BodyWeightSquats_g01_c01\\000'

In [32]:
train_ratio = 0.9
valid_ratio = 0.05
test_ratio = 0.05

for class_str in class_list:
    video_folder_list = glob(f"{common_sample_folder}/{class_str}/*")
    print(class_str, len(video_folder_list))


BodyWeightSquats 112
BoxingPunchingBag 163
BoxingSpeedBag 134
HandstandPushups 128
HighJump 123
JumpingJack 123
Lunges 127
PullUps 100
Punch 160
PushUps 100
WallPushups 130


In [23]:
len(video_folder_list)

391

# Filter Unused

In [21]:
common_sample_folder = "../data/1. Sampling npy"
target_sample_list = os.listdir("../data/0-2. Sampled/")

sample_folder_list = glob(f"{common_sample_folder}/*")

for sample_folder in sample_folder_list:
    is_in = False
    for target_sample in target_sample_list:
        if target_sample in sample_folder:
            is_in=True
            break
    if is_in:
        pass
    else:
        shutil.rmtree(sample_folder)

In [5]:
sample_array.shape

(8, 240, 320, 3)

In [15]:
list(range(0, total_frame - frame_per_sample, stride))

[0,
 6,
 12,
 18,
 24,
 30,
 36,
 42,
 48,
 54,
 60,
 66,
 72,
 78,
 84,
 90,
 96,
 102,
 108,
 114,
 120,
 126,
 132,
 138,
 144,
 150]

In [ ]:
video_array

In [19]:
video_array.shape

(164, 240, 320, 3)

In [14]:
photo_list[0].shape

(240, 320, 3)

In [12]:
video_array.shape

(164, 240, 320, 3)

In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from src.model.vision_transformer import swin_layers, transformer_layers

In [4]:
input_size = (28, 28, 1) # The image size of the MNIST
patch_size = (2, 2) # Segment 28-by-28 frames into 2-by-2 sized patches, patch contents and positions are embedded
n_labels = 10 # MNIST labels

# Dropout parameters
mlp_drop_rate = 0.01 # Droupout after each MLP layer
attn_drop_rate = 0.01 # Dropout after Swin-Attention
proj_drop_rate = 0.01 # Dropout at the end of each Swin-Attention block, i.e., after linear projections
drop_path_rate = 0.01 # Drop-path within skip-connections

# Self-attention parameters 
# (Fixed for all the blocks in this configuration, but can vary per block in larger architectures)
num_heads = 8 # Number of attention heads
embed_dim = 64 # Number of embedded dimensions
num_mlp = 256 # Number of MLP nodes
qkv_bias = True # Convert embedded patches to query, key, and values with a learnable additive value
qk_scale = None # None: Re-scale query based on embed dimensions per attention head # Float for user specified scaling factor

# Shift-window parameters
window_size = 2 # Size of attention window (height = width)
shift_size = window_size // 2 # Size of shifting (shift_size < window_size)

num_patch_x = input_size[0]//patch_size[0]
num_patch_y = input_size[1]//patch_size[1]

In [6]:
# The input section
IN = Inputs(input_size)
X = IN

# Extract patches from the input tensor
X = transformer_layers.patch_extract(patch_size)(X)

# Embed patches to tokens
X = transformer_layers.patch_embedding(num_patch_x*num_patch_y, embed_dim)(X)

# -------------------- Swin transformers -------------------- #
# Stage 1: window-attention + Swin-attention + patch-merging

for i in range(2):
    
    if i % 2 == 0:
        shift_size_temp = 0
    else:
        shift_size_temp = shift_size

    X = swin_layers.SwinTransformerBlock(dim=embed_dim, num_patch=(num_patch_x, num_patch_y), num_heads=num_heads, 
                             window_size=window_size, shift_size=shift_size_temp, num_mlp=num_mlp, qkv_bias=qkv_bias, qk_scale=qk_scale,
                             mlp_drop=mlp_drop_rate, attn_drop=attn_drop_rate, proj_drop=proj_drop_rate, drop_path_prob=drop_path_rate, 
                             name='swin_block{}'.format(i))(X)
# Patch-merging
#    Pooling patch sequences. Half the number of patches (skip every two patches) and double the embedded dimensions
X = transformer_layers.patch_merging((num_patch_x, num_patch_y), embed_dim=embed_dim, name='down{}'.format(i))(X)

# ----------------------------------------------------------- #

# Convert embedded tokens (2D) to vectors (1D)
X = GlobalAveragePooling1D()(X)

# The output section
OUT = Dense(n_labels, activation='softmax')(X)
# Model configuration
model = keras.models.Model(inputs=[IN,], outputs=[OUT,])

NameError: name 'Inputs' is not defined